In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import re
import nltk
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
stopword = set(stopwords.words('english'))
bad_words = pd.read_csv('bad-words.csv')
data = pd.read_csv('twitter.csv')
data["labels"] = data["class"].map({0: "It's a Hate Speech", 1: " Alert, you are using Offensive language", 2: "No hate speech"})
data = data[["tweet", "labels"]]

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text
data["tweet"] = data["tweet"].apply(clean)
x = np.array(data["tweet"])
y = np.array(data["labels"])
cv = CountVectorizer()
X = cv.fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

def predict_with_asterisks(user_input):
    t = clean(user_input)
    t = cv.transform([t])
    output = clf.predict(t)
    if output == 'No hate speech':
        return user_input
    else:
        for word in bad_words['jigaboo']:
            user_input = user_input.replace(word, '*' * len(word))
        return user_input

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abhishek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import joblib

In [7]:
joblib.dump(clf,"model.joblib")

['model.joblib']

In [10]:
joblib.dump(cv,"vector.joblib")

['vector.joblib']